In [5]:
import datetime
import sqlite3 as sql 
import requests
import re
import nltk
import json
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from config_wip import config
bearertoken=config['bear_token']
headers = {'Authorization': 'Bearer ' +
       bearertoken, 'Content-Type': 'application/json'}

In [32]:
con = sql.connect(config['database'])
cur = con.cursor()
cur.execute('''CREATE TABLE IF NOT EXISTS twitter_tweet_to_user (tweet_id INT, user_id INT)''')
cur.execute('''CREATE TABLE IF NOT EXISTS twitter_tweet (tweet_id INT, conersation_id INT, text TEXT, likes INT, retweets INT, replay INT, quote INT, created TEXT)''')
cur.execute('''CREATE TABLE IF NOT EXISTS twitter_nea (tweet_id INT, text TEXT, entity TEXT)''')
cur.execute('''CREATE TABLE IF NOT EXISTS twitter_tweet_mention (tweet_id INT, mention_id INT, mention_name TEXT)''')
cur.execute('''CREATE TABLE IF NOT EXISTS twitter_tweet_hash_tags (tweet_id INT, hash_tags TEXT)''')
cur.execute('''CREATE TABLE IF NOT EXISTS twitter_tweet_urls (tweet_id INT, url TEXT, title TEXT)''')
cur.execute('''CREATE TABLE IF NOT EXISTS twitter_tweet_conText_entity (tweet_id INT, entity_id INT)''')
cur.execute('''CREATE TABLE IF NOT EXISTS twitter_tweet_conText_domain (tweet_id INT, domain_id INT)''')
cur.execute('''CREATE TABLE IF NOT EXISTS twitter_tweet_conText_def (id INT, name TEXT, description TEXT, type TEXT, PRIMARY KEY(id,type)) ''')
con.commit()

In [90]:
def twitter_query(kw, headers, cur):
    date = datetime.datetime.now() - datetime.timedelta(days=1)
    date = date.strftime("%Y-%m-%dT%H:%M:%SZ") 
    kw = kw.replace(' ', '%20')
    BASE_URL = f"https://api.twitter.com/2/tweets/search/recent?query=({kw})%20is%3Averified%20lang%3Aen%20-is%3Aretweet&start_time={date}&tweet.fields=id,author_id,context_annotations,conversation_id,created_at,entities,lang,public_metrics,text"
    auth_response_QUERY = requests.get(BASE_URL,  headers=headers)
    auth_response_RESPONSE = json.loads(auth_response_QUERY.text)
    if 'errors' in list(auth_response_RESPONSE.keys()):
        print(auth_response_RESPONSE['errors'])
        break
    try:
        for j in auth_response_RESPONSE['data']:
            input_dict = {
            'lang' : None,
            'hashtags' : [],
            'url' : [],
            'title' : [],
            'tweet_domain' : [],
            'tweet_entity' : [],
            'tweet_mentions' : [],
            'conversation_id' : None,
            'tweet_id' : j['id'],
            'text' : j['text'].replace("'","''"),
            'author_id' : j['author_id'],
            'created_at' : j['created_at'],
            'public_metrics': j['public_metrics'],
            'nea' : []
            }
            if 'lang' in list(j.keys()):
                input_dict['lang'] = j['lang']
                if input_dict['lang'] != 'en':
                    continue
            if 'entities' in list(j.keys()):
                if 'hashtags' in list(j['entities'].keys()):
                    for i in j['entities']['hashtags']:
                        hash_query = f"INSERT INTO twitter_tweet_hash_tags VALUES ({input_dict['tweet_id']}, '{i['tag']}')"
                        cur.execute(hash_query)
                        con.commit()
                if 'urls' in list(j['entities'].keys()):
                    for i in j['entities']['urls']:
                        if 'unwound_url' in list(i.keys()):
                            title = i['title'].replace("'","''")
                            url_query = f"INSERT INTO twitter_tweet_urls VALUES ({input_dict['tweet_id']}, '{i['unwound_url']}', '{title}')"
                            cur.execute(url_query)
                            con.commit()
                if 'annotations' in list(j['entities'].keys()):
                    for i in j['entities']['annotations']:
                        nea_query = f"INSERT INTO twitter_nea VALUES ({input_dict['tweet_id']}, '{i['normalized_text']}', '{i['type']}')"
                        cur.execute(nea_query)
                        con.commit()
                
                if 'mentions' in list(j['entities'].keys()):
                    for i in j['entities']['mentions']:
                        mention_query = f"INSERT INTO twitter_tweet_mention VALUES ({input_dict['tweet_id']}, {i['id']}, '{i['username']}')"
                        cur.execute(mention_query)
                        con.commit()
            if 'context_annotations' in list(j.keys()):
                for i in j['context_annotations']:
                    d_name = i['domain']['name'].replace("'","''")
                    d_des = i['domain']['description'].replace("'","''")
                    conText_d = f"INSERT INTO twitter_tweet_conText_domain VALUES ({input_dict['tweet_id']}, {i['domain']['id']})"
                    conDef_d = f"INSERT INTO twitter_tweet_conText_def VALUES ({i['domain']['id']}, '{d_name}', '{d_des}', 'domain') ON CONFLICT(id,type) DO NOTHING"
                    conText_e = f"INSERT INTO twitter_tweet_conText_entity VALUES ({input_dict['tweet_id']}, {i['entity']['id']})"
                    e_name = i['entity']['name'].replace("'","''")
                    if 'description' not in  list(i.keys()):
                        e_des=  None
                    else:
                        e_des = i['entity']['description'].replace("'","''")
                    conDef_e = f"INSERT INTO twitter_tweet_conText_def VALUES ({i['entity']['id']}, '{e_name}', '{e_des}', 'entity') ON CONFLICT(id,type) DO NOTHING"
                    cur.execute(conText_d)
                    cur.execute(conDef_d)
                    cur.execute(conText_e)
                    cur.execute(conDef_e)
                    con.commit()
            if 'conversation_id' in list(j.keys()):
                input_dict['conversation_id'] = j['conversation_id']
            tweet_query = f"INSERT INTO twitter_tweet VALUES ({input_dict['tweet_id']},{input_dict['conversation_id']},'{input_dict['text']}', {input_dict['public_metrics']['like_count']}, {input_dict['public_metrics']['retweet_count']}, {input_dict['public_metrics']['reply_count']}, {input_dict['public_metrics']['quote_count']},'{input_dict['created_at']}')"
            user_tweet_join_query = f"INSERT INTO twitter_tweet_to_user VALUES ({input_dict['tweet_id']},{input_dict['author_id']})"
            cur.execute(tweet_query)
            cur.execute(user_tweet_join_query)
            con.commit()
    except Exception as e:
        print(e)


SyntaxError: 'break' outside loop (2254065819.py, line 10)

In [76]:
def get_sample(con, n=15, exact=False):
    sql_query = f"""SELECT * FROM NEA_Report
    WHERE label IN ('ORG','PERSON') 
    ORDER BY RANDOM() 
    LIMIT {n};"""
    df = pd.read_sql(sql_query,con)
    if exact:
        return ['"'+str(i.replace('\n', ' '))+'"' for i in df['nea_text'].to_list()]
    else:
        return [i.replace('\n', ' ') for i in df['nea_text'].to_list()]

In [81]:
sample = get_sample(con)

In [82]:
kw = ' OR '.join(sample)

In [91]:
twitter_query(kw, headers, cur)

[{'parameters': {' HENRY COLLEGE  Sullivan County  Bomb) is:verified lang:en -is:retweet': ['']}, 'message': 'The query parameter [ HENRY COLLEGE  Sullivan County  Bomb) is:verified lang:en -is:retweet] is not one of [query,start_time,end_time,since_id,until_id,max_results,next_token,pagination_token,sort_order,expansions,tweet.fields,media.fields,poll.fields,place.fields,user.fields]'}]


KeyError: 'data'

In [92]:
kw

'Herbert%20Nelson%20OR%20Daily%20Times%20OR%20CentralMaine.com%20OR%20Department’s%20OR%20Johnson%20OR%20the%20Ballad%20Health%20COPA%20OR%20Sioux%20%20Falls%20%20Police%20%20Department%20OR%20STEM%20OR%20THE%20USA%20TODAY%20OR%20Justice%20Department%20OR%20Ma%20OR%20Stockdale%20Highway%20OR%20Cuomo%20OR%20Carrie%20Napoleon%20OR%20EMORY%20&%20HENRY%20COLLEGE%20%20Sullivan%20County%20%20Bomb'

In [88]:
date = datetime.datetime.now() - datetime.timedelta(days=1)
date = date.strftime("%Y-%m-%dT%H:%M:%SZ") 
kw = kw.replace(' ', '%20')
BASE_URL = f"https://api.twitter.com/2/tweets/search/recent?query=({kw})%20is%3Averified%20lang%3Aen%20-is%3Aretweet&start_time={date}&tweet.fields=id,author_id,context_annotations,conversation_id,created_at,entities,lang,public_metrics,text"
auth_response_QUERY = requests.get(BASE_URL,  headers=headers)
auth_response_RESPONSE = json.loads(auth_response_QUERY.text)
if 'errors' in list(auth_response_RESPONSE.keys()):
    print(kw)
for j in auth_response_RESPONSE['data']:
    input_dict = {
    'lang' : None,
    'hashtags' : [],
    'url' : [],
    'title' : [],
    'tweet_domain' : [],
    'tweet_entity' : [],
    'tweet_mentions' : [],
    'conversation_id' : None,
    'tweet_id' : j['id'],
    'text' : j['text'].replace("'","''"),
    'author_id' : j['author_id'],
    'created_at' : j['created_at'],
    'public_metrics': j['public_metrics'],
    'nea' : []
    }
    if 'lang' in list(j.keys()):
        input_dict['lang'] = j['lang']
        if input_dict['lang'] != 'en':
            continue
    if 'entities' in list(j.keys()):
        if 'hashtags' in list(j['entities'].keys()):
            for i in j['entities']['hashtags']:
                hash_query = f"INSERT INTO twitter_tweet_hash_tags VALUES ({input_dict['tweet_id']}, '{i['tag']}')"
                cur.execute(hash_query)
                con.commit()
        if 'urls' in list(j['entities'].keys()):
            for i in j['entities']['urls']:
                if 'unwound_url' in list(i.keys()):
                    title = i['title'].replace("'","''")
                    url_query = f"INSERT INTO twitter_tweet_urls VALUES ({input_dict['tweet_id']}, '{i['unwound_url']}', '{title}')"
                    cur.execute(url_query)
                    con.commit()
        if 'annotations' in list(j['entities'].keys()):
            for i in j['entities']['annotations']:
                nea_query = f"INSERT INTO twitter_nea VALUES ({input_dict['tweet_id']}, '{i['normalized_text']}', '{i['type']}')"
                cur.execute(nea_query)
                con.commit()
        
        if 'mentions' in list(j['entities'].keys()):
            for i in j['entities']['mentions']:
                mention_query = f"INSERT INTO twitter_tweet_mention VALUES ({input_dict['tweet_id']}, {i['id']}, '{i['username']}')"
                cur.execute(mention_query)
                con.commit()
    if 'context_annotations' in list(j.keys()):
        for i in j['context_annotations']:
            d_name = i['domain']['name'].replace("'","''")
            d_des = i['domain']['description'].replace("'","''")
            conText_d = f"INSERT INTO twitter_tweet_conText_domain VALUES ({input_dict['tweet_id']}, {i['domain']['id']})"
            conDef_d = f"INSERT INTO twitter_tweet_conText_def VALUES ({i['domain']['id']}, '{d_name}', '{d_des}', 'domain') ON CONFLICT(id,type) DO NOTHING"
            conText_e = f"INSERT INTO twitter_tweet_conText_entity VALUES ({input_dict['tweet_id']}, {i['entity']['id']})"
            e_name = i['entity']['name'].replace("'","''")
            if 'description' not in  list(i.keys()):
                e_des=  None
            else:
                e_des = i['entity']['description'].replace("'","''")
            conDef_e = f"INSERT INTO twitter_tweet_conText_def VALUES ({i['entity']['id']}, '{e_name}', '{e_des}', 'entity') ON CONFLICT(id,type) DO NOTHING"
            cur.execute(conText_d)
            cur.execute(conDef_d)
            cur.execute(conText_e)
            cur.execute(conDef_e)
            con.commit()
    if 'conversation_id' in list(j.keys()):
        input_dict['conversation_id'] = j['conversation_id']
    tweet_query = f"INSERT INTO twitter_tweet VALUES ({input_dict['tweet_id']},{input_dict['conversation_id']},'{input_dict['text']}', {input_dict['public_metrics']['like_count']}, {input_dict['public_metrics']['retweet_count']}, {input_dict['public_metrics']['reply_count']}, {input_dict['public_metrics']['quote_count']},'{input_dict['created_at']}')"
    user_tweet_join_query = f"INSERT INTO twitter_tweet_to_user VALUES ({input_dict['tweet_id']},{input_dict['author_id']})"
    cur.execute(tweet_query)
    cur.execute(user_tweet_join_query)
    con.commit()

Herbert%20Nelson%20OR%20Daily%20Times%20OR%20CentralMaine.com%20OR%20Department’s%20OR%20Johnson%20OR%20the%20Ballad%20Health%20COPA%20OR%20Sioux%20%20Falls%20%20Police%20%20Department%20OR%20STEM%20OR%20THE%20USA%20TODAY%20OR%20Justice%20Department%20OR%20Ma%20OR%20Stockdale%20Highway%20OR%20Cuomo%20OR%20Carrie%20Napoleon%20OR%20EMORY%20&%20HENRY%20COLLEGE%20%20Sullivan%20County%20%20Bomb


KeyError: 'data'

In [89]:
list(auth_response_RESPONSE.keys())

['errors', 'title', 'detail', 'type']